## Supervised Machine Learning - Support Vector Machines (Galaxies vs. Star)

### Dependencies:

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import extract

In [2]:
columns = ["obj_ID",'alpha','delta', 'u', 'g', 'r', 'i', 'z', 'run_ID',
       'rerun_ID', 'cam_col', 'field_ID', 'spec_obj_ID', 'redshift',
       'plate', 'MJD', 'fiber_ID']
target = ["class"]

In [3]:
# # Create engine
# engine = create_engine("sqlite:///

In [4]:
# # reflect an existing database into a new model
# Base = automap_base()
# # reflect the tables
# Base.prepare(engine, reflect=True)
# # Save references to each table
# Measurement = Base.classes.measurement
# Station = Base.classes.station

In [5]:
# # Create our session (link) from Python to the DB
# session = Session(engine)

In [6]:
# Load the data
file_path = Path('Resources/star_galaxy.csv', index=False)
data_df = pd.read_csv(file_path)
# df = df.loc[:, columns].copy()
data_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237660e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543780e+18,GALAXY,0.634794,5812,56354,171
1,1.237660e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176010e+19,GALAXY,0.779136,10445,58158,427
2,1.237660e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237660e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030110e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891860e+18,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81034,1.237680e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.060000e+19,GALAXY,0.000000,9374,57749,438
81035,1.237680e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586350e+18,GALAXY,0.404895,7626,56934,866
81036,1.237670e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112010e+18,GALAXY,0.143366,2764,54535,74
81037,1.237660e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,GALAXY,0.455040,6751,56368,470


In [7]:
data_df.set_index(['spec_obj_ID'], inplace = True)

In [8]:
data_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,class,redshift,plate,MJD,fiber_ID
spec_obj_ID,,,,,,,,,,,,,,,,,
6.543780e+18,1.237660e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,GALAXY,0.634794,5812,56354,171
1.176010e+19,1.237660e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,GALAXY,0.779136,10445,58158,427
5.152200e+18,1.237660e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,GALAXY,0.644195,4576,55592,299
1.030110e+19,1.237660e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,GALAXY,0.932346,9149,58039,775
6.891860e+18,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.060000e+19,1.237680e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,GALAXY,0.000000,9374,57749,438
8.586350e+18,1.237680e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,GALAXY,0.404895,7626,56934,866
3.112010e+18,1.237670e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,GALAXY,0.143366,2764,54535,74


In [9]:
 new_data_df=data_df.drop(['alpha','delta','run_ID','rerun_ID', 'cam_col', 'field_ID', "obj_ID",'plate', 'MJD', 'fiber_ID'], axis = 1 )

In [10]:
new_data_df

,u,g,r,i,z,class,redshift
spec_obj_ID,,,,,,,
6.543780e+18,23.87882,22.27530,20.39501,19.16573,18.79371,GALAXY,0.634794
1.176010e+19,24.77759,22.83188,22.58444,21.16812,21.61427,GALAXY,0.779136
5.152200e+18,25.26307,22.66389,20.60976,19.34857,18.94827,GALAXY,0.644195
1.030110e+19,22.13682,23.77656,21.61162,20.50454,19.25010,GALAXY,0.932346
6.891860e+18,19.43718,17.58028,16.49747,15.97711,15.54461,GALAXY,0.116123
...,...,...,...,...,...,...,...
1.060000e+19,22.16759,22.97586,21.90404,21.30548,20.73569,GALAXY,0.000000
8.586350e+18,22.69118,22.38628,20.45003,19.75759,19.41526,GALAXY,0.404895
3.112010e+18,21.16916,19.26997,18.20428,17.69034,17.35221,GALAXY,0.143366


### Variable engineering

In [11]:
frequency = [1, 2, 3]

u_size_bins = [-10000, -6600,-3310, 33]
new_data_df["binned_u"] = pd.cut(new_data_df["u"], u_size_bins, labels = frequency)

g_size_bins = [-10000, -6600,-3310, 33]
new_data_df["binned_g"] = pd.cut(new_data_df["g"], g_size_bins, labels = frequency)

r_size_bins = [9,16,23, 30]
new_data_df["binned_r"] = pd.cut(new_data_df["r"], r_size_bins, labels = frequency)

i_size_bins = [9,17,25, 33]
new_data_df["binned_i"] = pd.cut(new_data_df["i"], i_size_bins, labels = frequency)

z_size_bins = [-10000, -6600,-3310, 30]
new_data_df["binned_z"] = pd.cut(new_data_df["z"], z_size_bins, labels = frequency)

new_data_df.sample(30)

,u,g,r,i,z,class,redshift,binned_u,binned_g,binned_r,binned_i,binned_z
spec_obj_ID,,,,,,,,,,,,
9.444320e+18,26.12359,24.24993,21.67814,20.37538,19.59131,GALAXY,0.708269,3,3,2,2,3
1.211480e+18,21.32939,20.42546,19.89253,19.66953,19.56869,GALAXY,0.228159,3,3,2,2,3
1.244250e+18,21.01982,20.48201,19.95132,19.81973,19.72587,GALAXY,0.369830,3,3,2,2,3
9.867510e+18,23.08865,22.36689,22.25639,22.36184,21.64104,GALAXY,1.210704,3,3,2,2,3
6.466220e+18,22.41440,21.55760,20.49684,19.65604,19.20274,GALAXY,0.616143,3,3,2,2,3
7.486340e+18,26.25668,22.57729,20.73031,19.67002,19.15208,GALAXY,0.577450,3,3,2,2,3
7.458140e+18,22.08660,21.60054,19.72585,18.81538,18.49185,GALAXY,0.456943,3,3,2,2,3
7.691270e+18,24.70261,22.22655,20.37915,19.48280,19.02157,GALAXY,0.463090,3,3,2,2,3
1.475020e+18,14.92536,13.56020,13.10489,12.83076,12.64798,GALAXY,0.006129,3,3,1,1,3


### Test and Train Data

In [12]:
# Create our features
X = new_data_df.drop('class', axis=1)

# Create our target
y = new_data_df['class']

print(X.shape)
print(y.shape)

(81039, 11)
(81039,)


In [13]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
Counter(y_train)

Counter({'GALAXY': 44584, 'STAR': 16195})

### Random Oversampling

In [14]:
# Resample the training data with the RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'GALAXY': 44584, 'STAR': 44584})

### Support Vector Machine (Galaxies vs. Star) - Oversampling

In [15]:
# Train the Support Vector Machine Model using the resampled data
model = SVC(kernel='linear')
model.fit(X_resampled, y_resampled)

SVC(kernel='linear')

In [16]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9923289146087073

In [17]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

     GALAXY       1.00      0.98      1.00      0.99      0.99      0.98     14861
       STAR       0.96      1.00      0.98      0.98      0.99      0.99      5399

avg / total       0.99      0.99      1.00      0.99      0.99      0.98     20260



In [18]:
# Display the confusion matrix
c= confusion_matrix(y_test,y_pred)
cmros = pd.DataFrame(c, index=["Actual - Galaxy", "Actual - Star"], columns=["Predicted - Galaxy", "Predicted - Star"])
cmros

,Predicted - Galaxy,Predicted - Star
Actual - Galaxy,14633,228
Actual - Star,0,5399


### Random Undersampling

In [19]:
# Resample the training data with the RandomUnderSampler
rus = RandomUnderSampler(random_state=1)
X_resampled1, y_resampled1 = rus.fit_resample(X_train, y_train)
Counter(y_resampled1)

Counter({'GALAXY': 16195, 'STAR': 16195})

### Support Vector Machine (Galaxies vs. Star) - Undersampling

In [20]:
model2 = SVC(kernel='linear')
model2.fit(X_resampled1, y_resampled1)

SVC(kernel='linear')

In [21]:
# Calculated the balanced accuracy score
y_pred1 = model2.predict(X_test)
balanced_accuracy_score(y_test, y_pred1)

0.9887288876926182

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred1))

                   pre       rec       spe        f1       geo       iba       sup

     GALAXY       1.00      0.98      1.00      0.99      0.99      0.98     14861
       STAR       0.94      1.00      0.98      0.97      0.99      0.98      5399

avg / total       0.98      0.98      0.99      0.98      0.99      0.98     20260



In [23]:
# Display the confusion matrix
c= confusion_matrix(y_test,y_pred1)
cmrus = pd.DataFrame(c, index=["Actual - Galaxy", "Actual - Star"], columns=["Predicted - Galaxy", "Predicted - Star"])
cmrus

,Predicted - Galaxy,Predicted - Star
Actual - Galaxy,14526,335
Actual - Star,0,5399
